In [2]:
import numpy as np
import minterpy as mp
import minterpy_integration as mint
import scipy.special as special

In [ ]:
class Chebyshev(nn.Module):
    def __init__(self, degree, d, k, o):
        super(Chebyshev, self).__init__()        
     
        self.input_dimension = d 
        self.rank = k
        self.output_dimension = o 
        self.degree = int(degree)
        self.T0 = nn.Parameter(nn.Linear(k, 1).weight.to(torch.float32)[0])
        for i in range(1, self.degree + 1):
            setattr(self, 'T{}'.format(i), nn.Linear(self.input_dimension, self.rank, bias=False)) 

        self.layer_C = nn.Linear(self.rank, self.output_dimension) 


    def forward(self, z):
        z = z.reshape(-1, self.input_dimension)
        out0 = self.T0
        out1 = self.T1(z)
        out_list = [out0] + [out1]
        for i in range(2, self.degree + 1, 2):
            out0 = getattr(self, 'T{}'.format(i))(z) * 2 * out1 - out0
            out_list = out_list + [out0]
            if i == self.degree:
                x = self.layer_C(out_list[self.degree])
                return x
            else:
                out1 = getattr(self, 'T{}'.format(i + 1))(z) * 2 * out0 - out1
                out_list = out_list + [out1]
        x = self.layer_C(out_list[self.degree])
        return x


In [3]:
def monomial_integral_cheb1st(exponents):
    cond = exponents>0
    term = np.ones(exponents.shape)*np.pi
    term[cond] = np.sqrt(np.pi)/exponents[cond] * ((-1)**exponents[cond] + 1)*special.gamma((exponents[cond] + 1)/2)/special.gamma(exponents[cond]/2)
    return np.prod(term,axis = 0)

def monomial_integral_cheb2nd(exponents):
    term = np.sqrt(np.pi)/4*((-1)**exponents + 1)*special.gamma((exponents + 1)/2)/special.gamma(exponents/2+2)
    return np.prod(term, axis = 0)

In [4]:
spatial_dimension = 3
poly_degree = 8
lp_degree = 2

In [5]:
multiindex = mp.MultiIndexSet.from_degree(spatial_dimension,poly_degree,lp_degree)

In [8]:
exponents = multiindex.exponents

In [6]:
cheb1 = monomial_integral_cheb1st(multiindex.exponents.T)

In [10]:
cheb1

array([31.00627668,  0.        , 15.50313834,  0.        , 11.62735376,
        0.        ,  9.68946146,  0.        ,  8.47827878,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , 15.50313834,  0.        ,  7.75156917,
        0.        ,  5.81367688,  0.        ,  4.84473073,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        , 11.62735376,  0.        ,
        5.81367688,  0.        ,  4.36025766,  0.        ,  3.63354805,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  9.68946146,  0.        ,  4.84473073,
        0.        ,  3.63354805,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  8.47827878,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.  